# Concurrency with asyncio
For real parallelism, you must have multiple cores. A modern laptop has four CPU cores but is routinely running more than 100 processes at any given time under normal, casual use. So, in practice, most processing happens concurrently and not in parallel.

This chapter introduces asyncio, a package that implements concurrency with coroutines driven by an event loop. It’s one of the largest and most ambitious libraries ever added to Python. Guido van Rossum developed asyncio outside of the Python repos‐
itory and gave the project a code name of 'Tulip' — so you’ll see references to that flower when researching this topic online.

## Thread Versus Coroutine: A Comparison


In [1]:
import threading
import itertools
import time
import sys

class Signal:
    go = True

def spin(msg, signal):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        time.sleep(.1)
        if not signal.go:
            break
    write(' ' * len(status) + '\x08' * len(status))

def slow_function():
    # pretend waiting a long time for I/O
    time.sleep(3)
    return 42

def supervisor():
    signal = Signal()
    spinner = threading.Thread(target=spin,args=('thinking!', signal))
    print('spinner object:', spinner)
    spinner.start()
    result = slow_function()
    signal.go = False
    spinner.join()
    return result

def main():
    result = supervisor()
    print('Answer:', result)

if __name__ == '__main__':
    main()


spinner object: <Thread(Thread-6, initial)>
| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking          Answer: 42


I also like the use of itertools cycle.

In [27]:
class A:
    def __init__(self):
        self.my_list = list(range(3))
        self.finished = False

    def __call__(self):
        if len(self.my_list) > 0:
            return self.my_list.pop()
        else:
            self.finished = True
            return None


In [28]:
import itertools

a = A()
b = A()
for obj in itertools.cycle([a, b]):
    value = obj()
    print(value)
    if a.finished and b.finished:
        break


2
2
1
1
0
0
None
None
